# Dependencias

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from scipy.stats import kurtosis, skew
from skimage.feature.texture import greycomatrix, greycoprops
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import datasets, svm, metrics, model_selection
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, mean_squared_error
from sklearn.neural_network import MLPClassifier, MLPRegressor
from pprint import pprint
import pickle

# Evaluación de modelos clásicos de aprendizaje supervisado

In [2]:
# Obtain the 7 requested stats from a given dataset
def extract_descriptors(data, size):
    out = []
    for image in data:
        max_val = image.max()
        mean_val = image.mean()
        var_val = image.var()
        skew_val = skew(image)
        kur_val = kurtosis(image)

        image = np.reshape(image, (size, size))

        tmp = (image-image.min()).astype(int)
        glcm = greycomatrix(tmp , distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
        dissimilarity = greycoprops(glcm, 'dissimilarity')[0, 0]
        correlation = greycoprops(glcm, 'correlation')[0, 0]
        out.append([max_val, mean_val, var_val, skew_val, kur_val, dissimilarity, correlation])
    return out

## Fashion M-Nist

In [4]:
# Extract and divide the dataset

(f_x_train, f_y_train), (f_x_test, f_y_test) = fashion_mnist.load_data()

f_x_train = f_x_train.reshape(-1, 784).astype("float32") / 255
f_x_test = f_x_test.reshape(-1, 784).astype("float32") / 255

f_x_train_desc = extract_descriptors(f_x_train, 28)
f_x_test_desc = extract_descriptors(f_x_test, 28)

In [22]:
fashion_scores = {"pixels": {}, "descriptors":{}}

In [23]:
# F-MNIST LDA Pixels
clf = LinearDiscriminantAnalysis()
clf.fit(f_x_train, f_y_train)
y_pred = clf.predict(f_x_test)
accuracy = clf.score(f_x_train, f_y_train)
recall = recall_score(f_y_test, y_pred, average=None)
fashion_scores["pixels"]["lda"] = {"accuracy": accuracy, "recall": recall}

In [24]:
# F-MNIST KNN Pixels
neigh = KNeighborsClassifier(n_neighbors=15)
neigh.fit(f_x_train, f_y_train)
y_pred = clf.predict(f_x_test)
accuracy = clf.score(f_x_train, f_y_train)
recall = recall_score(f_y_test, y_pred, average=None)
fashion_scores["pixels"]["knn"] = {"accuracy": accuracy, "recall": recall}

In [25]:
# F-MNIST LDA Descriptors
clf = LinearDiscriminantAnalysis()
clf.fit(f_x_train_desc, f_y_train)
y_pred = clf.predict(f_x_test_desc)
accuracy = clf.score(f_x_train_desc, f_y_train)
recall = recall_score(f_y_test, y_pred, average=None)
fashion_scores["descriptors"]["lda"] = {"accuracy": accuracy, "recall": recall}

In [26]:
# F-MNIST KNN Descriptors
neigh = KNeighborsClassifier(n_neighbors=15)
neigh.fit(f_x_train_desc, f_y_train)
y_pred = clf.predict(f_x_test_desc)
accuracy = clf.score(f_x_train_desc, f_y_train)
recall = recall_score(f_y_test, y_pred, average=None)
fashion_scores["descriptors"]["knn"] = {"accuracy": accuracy, "recall": recall}

In [27]:
pprint(fashion_scores)

{'descriptors': {'knn': {'accuracy': 0.3429333333333333,
                         'recall': array([0.123, 0.433, 0.305, 0.143, 0.38 , 0.555, 0.369, 0.469, 0.128,
       0.452])},
                 'lda': {'accuracy': 0.3429333333333333,
                         'recall': array([0.123, 0.433, 0.305, 0.143, 0.38 , 0.555, 0.369, 0.469, 0.128,
       0.452])}},
 'pixels': {'knn': {'accuracy': 0.8325833333333333,
                    'recall': array([0.777, 0.933, 0.682, 0.846, 0.743, 0.89 , 0.558, 0.893, 0.92 ,
       0.909])},
            'lda': {'accuracy': 0.8325833333333333,
                    'recall': array([0.777, 0.933, 0.682, 0.846, 0.743, 0.89 , 0.558, 0.893, 0.92 ,
       0.909])}}}


## Emojis

In [28]:
data = np.loadtxt("emojis.txt") 
x = data[:,1:]
y = data[:,0]

e_x_train, e_x_test, e_y_train, e_y_test = train_test_split(x, y, test_size=0.2)

e_x_train_desc = extract_descriptors(e_x_train, 32)
e_x_test_desc = extract_descriptors(e_x_test, 32)

In [29]:
emojis_scores = {"pixels": {}, "descriptors":{}}

In [30]:
# Emojis LDA Pixels
clf = LinearDiscriminantAnalysis()
score_cv = model_selection.cross_val_score(clf, e_x_train, e_y_train)
y_pred = model_selection.cross_val_predict(clf, e_x_test, e_y_test)
recall = recall_score(e_y_test, y_pred, average=None)
emojis_scores["pixels"]["lda"] = {"accuracy": max(score_cv), "recall": recall}

In [31]:
# Emojis KNN Pixels
neigh = KNeighborsClassifier(n_neighbors=15)
score_cv = model_selection.cross_val_score(neigh, e_x_train, e_y_train)
y_pred = model_selection.cross_val_predict(neigh, e_x_test, e_y_test)
recall = recall_score(e_y_test, y_pred, average=None)
emojis_scores["pixels"]["knn"] = {"accuracy": max(score_cv), "recall": recall}

In [32]:
# Emojis LDA Descriptors
clf = LinearDiscriminantAnalysis()
score_cv = model_selection.cross_val_score(clf, e_x_train_desc, e_y_train)
y_pred = model_selection.cross_val_predict(clf, e_x_test_desc, e_y_test)
recall = recall_score(e_y_test, y_pred, average=None)
emojis_scores["descriptors"]["lda"] = {"accuracy": accuracy, "recall": recall}

In [33]:
# Emojis KNN Descriptors
neigh = KNeighborsClassifier(n_neighbors=15)
score_cv = model_selection.cross_val_score(neigh, e_x_train_desc, e_y_train)
y_pred = model_selection.cross_val_predict(neigh, e_x_test_desc, e_y_test)
recall = recall_score(e_y_test, y_pred, average=None)
emojis_scores["descriptors"]["knn"] = {"accuracy": max(score_cv), "recall": recall}

In [34]:
pprint(emojis_scores)

{'descriptors': {'knn': {'accuracy': 0.3341772151898734,
                         'recall': array([0.32258065, 0.35294118, 0.4494382 , 0.20833333, 0.25438596])},
                 'lda': {'accuracy': 0.3429333333333333,
                         'recall': array([0.27956989, 0.24509804, 0.70786517, 0.26041667, 0.33333333])}},
 'pixels': {'knn': {'accuracy': 0.6531645569620254,
                    'recall': array([0.27956989, 0.48039216, 0.97752809, 0.5       , 0.42982456])},
            'lda': {'accuracy': 0.6506329113924051,
                    'recall': array([0.55913978, 0.53921569, 1.        , 0.39583333, 0.56140351])}}}


## Marte

In [35]:
inputFile = open('navigability.obj', 'rb')
data = pickle.load(inputFile)

In [36]:
# Obtain the 7 requested stats from a given dataset
def mars_extract_descriptors(data, size):
    out = []
    for images in data:
        temp = []
        for image in images:
            image = image.flatten()
            
            max_val = image.max()
            mean_val = image.mean()
            var_val = image.var()
            skew_val = skew(image)
            kur_val = kurtosis(image)
            
            temp.extend([max_val, mean_val, var_val, skew_val, kur_val])
        out.append(temp)
    return out

In [37]:
x = [i[-4:] for i in data]
y = [j[0] for j in data]

m_x_train, m_x_test, m_y_train, m_y_test = train_test_split(x, y, test_size=0.2)

m_x_train_desc = mars_extract_descriptors(m_x_train, 20)
m_x_test_desc = mars_extract_descriptors(m_x_test, 20)

In [38]:
mars_scores = {}

In [58]:
reg = KNeighborsRegressor(n_neighbors=15)
score_cv = model_selection.cross_val_score(reg, m_x_train_desc, m_y_train)
y_pred = model_selection.cross_val_predict(reg, m_x_test_desc, m_y_test)
mse = mean_squared_error(m_y_test, y_pred)
mars_scores["knn"] = {"accuracy": max(score_cv), "mse": mse}

In [59]:
mars_scores

{'knn': {'accuracy': 0.5717949609035622, 'mse': 0.8732026143790851}}

# Clasificación con modelos perceptrón multicapa


## Fashion M-Nist

In [41]:
mlp = MLPClassifier(max_iter=100)
mlp.fit(f_x_train, f_y_train)
y_pred = mlp.predict(f_x_test)
accuracy = mlp.score(f_x_train, f_y_train)
recall = recall_score(f_y_test, y_pred, average=None)
fashion_scores["pixels"]["mlp"] = {"accuracy": accuracy, "recall": recall}

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [42]:
mlp = MLPClassifier(max_iter=100)
mlp.fit(f_x_train_desc, f_y_train)
y_pred = mlp.predict(f_x_test_desc)
accuracy = mlp.score(f_x_train_desc, f_y_train)
recall = recall_score(f_y_test, y_pred, average=None)
fashion_scores["descriptors"]["mlp"] = {"accuracy": accuracy, "recall": recall}

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [43]:
pprint(fashion_scores)

{'descriptors': {'knn': {'accuracy': 0.3429333333333333,
                         'recall': array([0.123, 0.433, 0.305, 0.143, 0.38 , 0.555, 0.369, 0.469, 0.128,
       0.452])},
                 'lda': {'accuracy': 0.3429333333333333,
                         'recall': array([0.123, 0.433, 0.305, 0.143, 0.38 , 0.555, 0.369, 0.469, 0.128,
       0.452])},
                 'mlp': {'accuracy': 0.4014666666666667,
                         'recall': array([0.342, 0.565, 0.317, 0.234, 0.36 , 0.517, 0.37 , 0.509, 0.189,
       0.601])}},
 'pixels': {'knn': {'accuracy': 0.8325833333333333,
                    'recall': array([0.777, 0.933, 0.682, 0.846, 0.743, 0.89 , 0.558, 0.893, 0.92 ,
       0.909])},
            'lda': {'accuracy': 0.8325833333333333,
                    'recall': array([0.777, 0.933, 0.682, 0.846, 0.743, 0.89 , 0.558, 0.893, 0.92 ,
       0.909])},
            'mlp': {'accuracy': 0.9789833333333333,
                    'recall': array([0.811, 0.975, 0.775, 0.892, 0.849, 

## Emojis

In [53]:
mlp = MLPClassifier(max_iter=100)
score_cv = model_selection.cross_val_score(mlp, e_x_train, e_y_train)
y_pred = model_selection.cross_val_predict(mlp, e_x_test, e_y_test)
recall = recall_score(e_y_test, y_pred, average=None)
emojis_scores["pixels"]["mlp"] = {"accuracy": max(score_cv), "recall": recall}

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

In [54]:
mlp = MLPClassifier(max_iter=100)
score_cv = model_selection.cross_val_score(mlp, e_x_train_desc, e_y_train)
y_pred = model_selection.cross_val_predict(mlp, e_x_test_desc, e_y_test)
recall = recall_score(e_y_test, y_pred, average=None)
emojis_scores["descriptors"]["mlp"] = {"accuracy": max(score_cv), "recall": recall}

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

In [55]:
pprint(emojis_scores)

{'descriptors': {'knn': {'accuracy': 0.3341772151898734,
                         'recall': array([0.32258065, 0.35294118, 0.4494382 , 0.20833333, 0.25438596])},
                 'lda': {'accuracy': 0.3429333333333333,
                         'recall': array([0.27956989, 0.24509804, 0.70786517, 0.26041667, 0.33333333])},
                 'mlp': {'accuracy': 0.37721518987341773,
                         'recall': array([0.        , 0.09803922, 0.61797753, 0.10416667, 0.55263158])}},
 'pixels': {'knn': {'accuracy': 0.6531645569620254,
                    'recall': array([0.27956989, 0.48039216, 0.97752809, 0.5       , 0.42982456])},
            'lda': {'accuracy': 0.6506329113924051,
                    'recall': array([0.55913978, 0.53921569, 1.        , 0.39583333, 0.56140351])},
            'mlp': {'accuracy': 0.8329113924050633,
                    'recall': array([0.5483871 , 0.71568627, 0.98876404, 0.53125   , 0.74561404])}}}


## Marte

In [64]:
reg = MLPRegressor(max_iter=1000)
score_cv = model_selection.cross_val_score(reg, m_x_train_desc, m_y_train)
y_pred = model_selection.cross_val_predict(reg, m_x_test_desc, m_y_test)
mse = mean_squared_error(m_y_test, y_pred)
mars_scores["mlp"] = {"accuracy": max(score_cv), "mse": mse}

In [65]:
mars_scores

{'knn': {'accuracy': 0.5717949609035622, 'mse': 0.8732026143790851},
 'mlp': {'accuracy': -1.249157202691706, 'mse': 2350.117069057886}}